**CMP585 - Structural Bioinformatics**

Cristian Lopes

*Assignment 2*

In [1]:
import pandas as pd
import numpy as np

# Load Data

In [2]:
def read_molecule(path):
    atoms = []
    with open(path, 'r') as f:
        for line in f:
            if line.startswith('ATOM'):
                atoms.append({'atom_name': line[12:16].strip(), 
                             'residue_name': line[17:20].strip(),
                             'residue_seq_number': line[22:26].strip(),
                             'x': float(line[30:38].strip()), 
                             'y': float(line[38:46].strip()),
                             'z': float(line[46:54].strip())})
    return pd.DataFrame(atoms)

In [3]:
molecule = read_molecule('./data/1crn.pdb')

In [4]:
molecule.head()

,atom_name,residue_name,residue_seq_number,x,y,z
0,N,THR,1,17.047,14.099,3.625
1,CA,THR,1,16.967,12.784,4.338
2,C,THR,1,15.685,12.755,5.133
3,O,THR,1,15.268,13.825,5.594
4,CB,THR,1,18.170,12.703,5.337


# Q1) Secondary to Tertiary Structure

## Symbol of each residue

First of all, we define the symbol of each residue.

In [5]:
residue_name_to_symbol = {
    'THR': 'T', 'CYS': 'C', 'PRO': 'P', 'SER': 'S', 'ILE': 'I', 'VAL': 'V',
    'ALA': 'A', 'ARG': 'R', 'ASN': 'N', 'PHE': 'F', 'LEU': 'L', 'GLY': 'G',
    'GLU': 'E', 'ASP': 'D'
}

## Position of each atom inside the residue 

For estimating the position of each atom inside the residues, we take the mean of the positions of each atom for each residue type.

In [6]:
residues = pd.DataFrame(molecule.groupby(by=['residue_name', 'atom_name']).mean())

In [7]:
residues.head(20)

x          y         z
residue_name atom_name                                
ALA          C          10.692800  10.906400  8.351600
             CA         11.111000  11.299000  8.257400
             CB         11.686600  11.816600  8.384800
             N          10.924600  11.246600  8.011000
             O          10.664800  10.522600  8.547600
ARG          C           6.768000   4.584500  2.733500
             CA          7.195500   5.549000  3.812000
             CB          8.608500   5.263500  4.287000
             CD          9.587500   6.823000  3.314500
             CG          9.234500   5.647000  4.177500
             CZ         11.147500   8.071000  2.506000
             N           6.322000   5.568000  4.952500
             NE         10.677500   7.213500  3.349000
             NH1        10.599000   8.641000  1.563000
             NH2        12.197000   8.406500  2.651500
             O           7.053000   4.839500  1.572000
ASN          C           7.844667   4.638000  6.353667
             CA          8.129333   4.312333  7.473333
             CB          7.691333   3.950000  8.163000
             CG          7.965000   3.631667  9.087333

Then, we move all the atoms of each residue type to the reference, considering the Alpha Carbon as system reference for each residue type.

In [8]:
residues.x -= residues.unstack(level=1).x['CA']
residues.y -= residues.unstack(level=1).y['CA']
residues.z -= residues.unstack(level=1).z['CA']

In [9]:
residues.head(20)

x         y         z
residue_name atom_name                              
ALA          C         -0.418200 -0.392600  0.094200
             CA         0.000000  0.000000  0.000000
             CB         0.575600  0.517600  0.127400
             N         -0.186400 -0.052400 -0.246400
             O         -0.446200 -0.776400  0.290200
ARG          C         -0.427500 -0.964500 -1.078500
             CA         0.000000  0.000000  0.000000
             CB         1.413000 -0.285500  0.475000
             CD         2.392000  1.274000 -0.497500
             CG         2.039000  0.098000  0.365500
             CZ         3.952000  2.522000 -1.306000
             N         -0.873500  0.019000  1.140500
             NE         3.482000  1.664500 -0.463000
             NH1        3.403500  3.092000 -2.249000
             NH2        5.001500  2.857500 -1.160500
             O         -0.142500 -0.709500 -2.240000
ASN          C         -0.284667  0.325667 -1.119667
             CA         0.000000  0.000000  0.000000
             CB        -0.438000 -0.362333  0.689667
             CG        -0.164333 -0.680667  1.614000

Right now, we have a dictionnary of residues containing the position of each atom inside the residue. We are not taking 

Next step is to replace each symbol of the secondary structure (e.g. TTCCPSIVARSNFNVCRLPGTPEAICATYTGCIIIPGATCPGDYAN) by its corresponding residue. We should also find a way to calculate the translation that should be applied to each residue in order to form the peptide bond. Idea: set the distance of N-C_alpha and ignore the rotation of each residue.

Done!